In [1]:
%cd "P:/projects/datajoint_wahl"
import login
login.connect()
import pathlib
import json
import pickle
from tqdm import tqdm

P:\projects\datajoint_wahl
Connecting mpanze@130.60.53.47:3306


In [9]:
login.set_working_directory("F:/Jithin")
print(login.get_working_directory())
p_base = pathlib.Path("F:/Jithin")
x = 0
from schema.common_exp import Session
from schema.mpanze_widefield import Scan, ScanInfo, RawImagingFile, AffineRegistration
from schema.mpanze_mapping import MappingSession, RawSynchronisationFile, RawParameterFile

def get_entries(p_b):
    d = {}
    suf = ["scan", "info", "img", "session", "affine"]
    for i in p_b:
        for j in suf:
            if j in str(i):
                with open(i,"rb") as f:
                    d[j] = pickle.load(f)
    return d

for p_json in tqdm(p_base.rglob("*.json")):
    if ("Widefield" in str(p_json.parent) and "M41" in str(p_json.parent)):
        continue
    p_tif = p_json.with_suffix(".tif")
    p_txt = p_json.with_suffix(".txt")
    p_backups = []
    if "orelimb" in p_json.stem:
        p_backups = list(p_json.parent.rglob("*orelimb*.pickle"))
    elif "indlimb" in p_json.stem:
        p_backups = list(p_json.parent.rglob("*indlimb*.pickle"))
    else:
        continue
    
    d = get_entries(p_backups)
    # modify session entry dict
    del d["session"]["session_num"]
    del d["session"]["session_id"]
    del d["session"]["session_counter"]
    del d["session"]["session_path"]
    d_restr = {"username":d["session"]["username"], "mouse_id":d["session"]["mouse_id"], "day":d["session"]["day"]}
    sess_count = len(Session() & d_restr) + 1
    d["session"]["session_num"] = sess_count
    d["session"]["session_path"] = p_json.parent
    Session().helper_insert1(d["session"])
    del d["session"]["session_path"]
    
    d["scan"]["session_num"] = sess_count
    d["scan"]["scan_id"] = 1
    Scan().insert1(d["scan"])
    d["info"]["session_num"] = sess_count
    d["info"]["scan_id"] = 1
    ScanInfo().insert1(d["info"])
    d["img"]["session_num"] = sess_count
    d["img"]["scan_id"] = 1
    d["img"]["filename_img"] = p_tif.stem + p_tif.suffix
    RawImagingFile().insert1(d["img"])
    if "affine" in d:
        d["affine"]["session_num"] = sess_count
        d["affine"]["scan_id"] = 1
        AffineRegistration().insert1(d["affine"])
    d_restr["session_num"] = sess_count
    d_mapping = {**d_restr, "stim_config": 0}
    if "orelimb" in p_json.stem:
        d_mapping["stim_bodypart"] = "forelimb_right"
    elif "indlimb" in p_json.stem:
        d_mapping["stim_bodypart"] = "hindlimb_right"
    else:
        raise Exception("+++++++++++")
    MappingSession().insert1(d_mapping)
    d_sync = {**d_restr, "filename_sync": p_txt.stem + p_txt.suffix}
    d_param = {**d_restr, "filename_params": p_json.stem + p_json.suffix}
    RawSynchronisationFile().insert1(d_sync)
    RawParameterFile().insert1(d_param)
    
    # insert data in pipeline
    if len(p_backups) == 0:
        print(len(p_backups))
    x +=1
    if p_tif.exists() and p_txt.exists():
        continue
    
assert x==80

0it [00:00, ?it/s]

F:/Jithin


96it [00:34,  2.78it/s]


In [ ]:
p_tif.stem + p_tif.suffix

In [ ]:
d["img"]

In [5]:
login.set_working_directory("F:/Jithin")